In [1]:
import tensorflow as tf
import cv2 
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# Configuaration of path and data
Datadirectory = "train/"
Classes = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
img_size = 224
batch_size = 100

In [3]:
img_size = 224
training_Data = [] 

In [4]:
#converting images to array :))

def create_training_Data():
    training_Data = []
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array, class_num])
            except Exception as e:
                pass
    return training_Data

In [5]:
training_Data = create_training_Data()
print(f"Total training samples: {len(training_Data)}")


Total training samples: 28709


In [6]:
random.shuffle(training_Data)

In [7]:
def split_data(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

In [8]:
def process_batch(batch):
    X = []
    Y = []
    for features, label in batch:
        X.append(features)
        Y.append(label)
    X = np.array(X).reshape(-1, img_size, img_size, 3)
    X = X.astype('float32') / 255.0
    Y = np.array(Y)
    return X, Y

In [9]:
for batch in split_data(training_Data, batch_size):
    X, Y = process_batch(batch)
    print(f"Processed batch with shape: {X.shape}, labels: {Y.shape}")

Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels: (100,)
Processed batch with shape: (100, 224, 224, 3), labels:

In [10]:
from tensorflow import keras
from tensorflow.keras import layers # type: ignore

In [11]:
base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [12]:
base_model.trainable = False

In [13]:
x = base_model.output
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
final_output = layers.Dense(7, activation='softmax')(x)  # 7 klas

In [14]:
new_model = keras.Model(inputs=base_model.input, outputs=final_output)

In [15]:
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
epochs = 10  # Liczba epok, dostosuj według potrzeb
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for batch in split_data(training_Data, batch_size):
        X, Y = process_batch(batch)
        loss, accuracy = new_model.train_on_batch(X, Y)
        print(f"Batch - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/10
Batch - Loss: 2.3641204833984375, Accuracy: 0.07000000029802322
Batch - Loss: 2.184861421585083, Accuracy: 0.0949999988079071
Batch - Loss: 2.0604679584503174, Accuracy: 0.1433333307504654
Batch - Loss: 2.003385305404663, Accuracy: 0.17000000178813934
Batch - Loss: 1.9766550064086914, Accuracy: 0.18199999630451202
Batch - Loss: 1.9416042566299438, Accuracy: 0.20000000298023224
Batch - Loss: 1.9232347011566162, Accuracy: 0.2085714340209961
Batch - Loss: 1.9141724109649658, Accuracy: 0.2199999988079071
Batch - Loss: 1.9034943580627441, Accuracy: 0.22333332896232605
Batch - Loss: 1.8960139751434326, Accuracy: 0.23100000619888306
Batch - Loss: 1.8845423460006714, Accuracy: 0.23181818425655365
Batch - Loss: 1.8682576417922974, Accuracy: 0.23749999701976776
Batch - Loss: 1.8609458208084106, Accuracy: 0.24076923727989197
Batch - Loss: 1.8497865200042725, Accuracy: 0.24857142567634583
Batch - Loss: 1.8415900468826294, Accuracy: 0.25
Batch - Loss: 1.8398480415344238, Accuracy: 0.2462

MemoryError: Unable to allocate 57.4 MiB for an array with shape (100, 224, 224, 3) and data type float32